# Merging data

Comme nous pouvons le voir, nous disposons de 4 fichiers représentant l'ensemble des informations concernant chaque accident.

Cependant, afin de pouvoir faire des analyses sur toutes ces différents attributs (features) il nous faut les rassembler en une seule table de donnée. C'est donc ce que nous allons faire dans ce notebook qui nous servira de pas-à-pas. Suite à cela nous pourrons retrouver ce module dans un script python séparé qui reprendra les éléments essentiels enn quelques méthodes.


## Imports

Avans de commencer, importons les librairies nécessaires à notre préprocessing.

In [431]:
# En premier, faisont les import dont je vais avoir besoin.
import pandas as pd
from pprint import pp
from collections import Counter
pd.__version__

'2.2.2'

## File retrieving

Now that we properly imported all the necessary libraries we need to retrieve the data contained in the files

To start with, we need their filepath. Luckily enough, the naming of our files is standardized, hence we can call each file with a single variable for the year. 

In [432]:

YEAR = "2022"

# We declare and intitialize a dictionnary containing all the files locations.
data_fpaths = {
    'lieux': "./data/" + YEAR + "/lieux-" + YEAR + ".csv",
    'vehicules': "./data/" + YEAR + "/vehicules-" + YEAR + ".csv",
    'carcteristiques': "./data/" + YEAR + "/carcteristiques-" + YEAR + ".csv",
    'usagers': "./data/" + YEAR + "/usagers-" + YEAR + ".csv"
}

# A simple pretty print to ensure that everything went alright.
pp(data_fpaths)

{'lieux': './data/2022/lieux-2022.csv',
 'vehicules': './data/2022/vehicules-2022.csv',
 'carcteristiques': './data/2022/carcteristiques-2022.csv',
 'usagers': './data/2022/usagers-2022.csv'}


## Quick Observation

Now let's take a quick look at our files to see what they are made of. I am volontarily not going to display all of them, but instead focus on the `usagers` file to save some space. But we could easily do the same thing for every other one.

We can observe that the column `Num_Acc` acts as a primary key.

In [433]:

data = { key : pd.read_csv(val, sep=';') for key, val in data_fpaths.items() }

data["usagers"].head()


C:\Users\user3\AppData\Local\Temp\ipykernel_18456\1080675184.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = { key : pd.read_csv(val, sep=';') for key, val in data_fpaths.items() }


,Num_Acc,id_usager,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,secu1,secu2,secu3,locp,actp,etatp
0,202200000001,1 099 700,813 952,A01,1,1,3,1,2008.0,5,2,8,-1,-1,-1,-1
1,202200000001,1 099 701,813 953,B01,1,1,1,1,1948.0,5,1,8,-1,-1,-1,-1
2,202200000002,1 099 698,813 950,B01,1,1,4,1,1988.0,9,1,0,-1,0,0,-1
3,202200000002,1 099 699,813 951,A01,1,1,1,1,1970.0,4,1,0,-1,0,0,-1
4,202200000003,1 099 696,813 948,A01,1,1,1,1,2002.0,0,1,0,-1,-1,-1,-1


In [434]:
# Some data description not used anymore but kept for archiving to show some possibilities of what we can do.
#data["usagers"].info()
#data["usagers"].describe()
#data["usagers"].hist()
#print(125296/126662)
#Counter(data["usagers"]["secu3"])

## Préparation des données

Maintenant qu'on a un peu explorer nos données (surtout les données usagers pour commencer), il faut rassembler nos quatres fichiers en un.

Pourquoi faire cela ? Eh bien tout simplement car on peut avoir des features dans d'autres fichiers qui sont utiles à la classification de nos données. Avoir plus de features peut permettre de récupérer plus d'information et d'avoir une meilleur prédiction.

Commençons donc par initialiser notre dataframe qui va rassembler toutes nos données

### Usagers

Take a look at `usagers`. 

In [435]:
data["usagers"]

,Num_Acc,id_usager,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,secu1,secu2,secu3,locp,actp,etatp
0,202200000001,1 099 700,813 952,A01,1,1,3,1,2008.0,5,2,8,-1,-1,-1,-1
1,202200000001,1 099 701,813 953,B01,1,1,1,1,1948.0,5,1,8,-1,-1,-1,-1
2,202200000002,1 099 698,813 950,B01,1,1,4,1,1988.0,9,1,0,-1,0,0,-1
3,202200000002,1 099 699,813 951,A01,1,1,1,1,1970.0,4,1,0,-1,0,0,-1
4,202200000003,1 099 696,813 948,A01,1,1,1,1,2002.0,0,1,0,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126657,202200055301,968 230,715 631,A01,1,1,1,2,2002.0,5,1,-1,-1,0,0,-1
126658,202200055301,968 231,715 631,A01,8,2,3,2,2004.0,5,1,-1,-1,0,0,-1
126659,202200055301,968 232,715 632,B01,1,1,4,2,1953.0,5,1,-1,-1,0,0,-1
126660,202200055302,968 228,715 629,A01,1,1,3,1,1992.0,1,2,6,-1,-1,-1,-1


In [436]:
pp(data["usagers"].info())
pp(data["usagers"].describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126662 entries, 0 to 126661
Data columns (total 16 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Num_Acc      126662 non-null  int64  
 1   id_usager    126662 non-null  object 
 2   id_vehicule  126662 non-null  object 
 3   num_veh      126662 non-null  object 
 4   place        126662 non-null  int64  
 5   catu         126662 non-null  int64  
 6   grav         126662 non-null  int64  
 7   sexe         126662 non-null  int64  
 8   an_nais      123788 non-null  float64
 9   trajet       126662 non-null  int64  
 10  secu1        126662 non-null  int64  
 11  secu2        126662 non-null  int64  
 12  secu3        126662 non-null  int64  
 13  locp         126662 non-null  int64  
 14  actp         126662 non-null  object 
 15  etatp        126662 non-null  int64  
dtypes: float64(1), int64(11), object(4)
memory usage: 15.5+ MB
None
            Num_Acc          place           c

Comme on peut le voir ci-dessous, on a trois colonne que l'on peut supprimer:
- **id_usager** : d'après la documentation en ligne, cette colonne n'apprait que dans le fichier `usagers` et ne contient que des valeurs uniques (permettant d'identifier de manière unique chaque usager). Cela n'apporte donc aucune information sur l'accident, ses causes ou sa gravité. Nous pouvons donc supprimer cette colonne.
- **id_vehicule** : Pareil ici, `id_vehicule` n'apporte aucune information sur l'accident
- **num_veh** : ceci est un identifiant propre à chaque accident pour identifier chaque vehicule impliqués. Mais il est repris entre différents accidents, on peut donc le supprimer également car il ne sert à rien pour notre analyse.

Aussi, il y a deux colonnes que l'on peut modifier:
- **an_nais** : Cette colonne ne contient que des années à 4 chiffres, on peut donc les traiter comme des entiers. Et les années non renseignées peuvent être misent à -1.
- **actp** : il s'agit d'une valeur à un chiffre en hexadécimale entre -1 et B. On peut simplement convertir cela en entier pour s'éviter d'avoir à manipuler des chaines de caractères. 

En plus, à l'exception de la première colonne qui sert de clé primaire, toutes les données peuvent être stockées sur 8bits. Donc nous pouvons les convertir en entier 8bit.

In [437]:
print(f"Nombre de valeurs uniques dans `id_usager` : {data['usagers']['id_usager'].nunique()}")
print(f"Nombre de valeurs uniques dans `id_vehicule` : {data['usagers']['id_vehicule'].nunique()}")
print(f"Nombre de valeurs uniques dans `num_veh` : {data['usagers']['num_veh'].nunique()}")
print(f"Nombre de valeurs nulles dans `an_nais` : {data['usagers']['an_nais'].isna().sum()}")
print(f"Type de `actp` : {data['usagers']['actp'].dtype}")

Nombre de valeurs uniques dans `id_usager` : 126662
Nombre de valeurs uniques dans `id_vehicule` : 94493
Nombre de valeurs uniques dans `num_veh` : 47
Nombre de valeurs nulles dans `an_nais` : 2874
Type de `actp` : object


In [438]:
YEAR = 2022
# First, drop the columns we don't need anymore
data["usagers"].drop(columns=["id_usager", "num_veh"], inplace=True)
# Then take care of the an_nais column by converting the year to the age and filling the null value by -1
data["usagers"]["an_nais"] = data["usagers"]["an_nais"].fillna(YEAR+1).apply(lambda x: YEAR - x)
data["usagers"].rename(columns={"an_nais":"age"}, inplace=True)
# Convert the hexadecimal value of actp to a decimal value
data["usagers"]["actp"] = data["usagers"]["actp"].apply(lambda x: int(str(x), 16))
# Change the type of every column to int8 except the first one.
data["usagers"] = data["usagers"].astype({col: 'int8' for col in data["usagers"].columns if col not in ("id_vehicule", 'Num_Acc')})
# Check what we've done.
data["usagers"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126662 entries, 0 to 126661
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Num_Acc      126662 non-null  int64 
 1   id_vehicule  126662 non-null  object
 2   place        126662 non-null  int8  
 3   catu         126662 non-null  int8  
 4   grav         126662 non-null  int8  
 5   sexe         126662 non-null  int8  
 6   age          126662 non-null  int8  
 7   trajet       126662 non-null  int8  
 8   secu1        126662 non-null  int8  
 9   secu2        126662 non-null  int8  
 10  secu3        126662 non-null  int8  
 11  locp         126662 non-null  int8  
 12  actp         126662 non-null  int8  
 13  etatp        126662 non-null  int8  
dtypes: int64(1), int8(12), object(1)
memory usage: 3.4+ MB


In [439]:
data["usagers"]

,Num_Acc,id_vehicule,place,catu,grav,sexe,age,trajet,secu1,secu2,secu3,locp,actp,etatp
0,202200000001,813 952,1,1,3,1,14,5,2,8,-1,-1,-1,-1
1,202200000001,813 953,1,1,1,1,74,5,1,8,-1,-1,-1,-1
2,202200000002,813 950,1,1,4,1,34,9,1,0,-1,0,0,-1
3,202200000002,813 951,1,1,1,1,52,4,1,0,-1,0,0,-1
4,202200000003,813 948,1,1,1,1,20,0,1,0,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126657,202200055301,715 631,1,1,1,2,20,5,1,-1,-1,0,0,-1
126658,202200055301,715 631,8,2,3,2,18,5,1,-1,-1,0,0,-1
126659,202200055301,715 632,1,1,4,2,69,5,1,-1,-1,0,0,-1
126660,202200055302,715 629,1,1,3,1,30,1,2,6,-1,-1,-1,-1


### Vehicules

Maintenant regardons le fichier `vehicules`

In [440]:
data["vehicules"]

,Num_Acc,id_vehicule,num_veh,senc,catv,obs,obsm,choc,manv,motor,occutc
0,202200000001,813 952,A01,1,2,0,2,1,9,1,NaN
1,202200000001,813 953,B01,1,7,0,2,2,1,1,NaN
2,202200000002,813 950,B01,2,7,0,2,8,15,1,NaN
3,202200000002,813 951,A01,2,10,0,2,1,1,1,NaN
4,202200000003,813 948,A01,2,7,0,2,1,2,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...
94488,202200055300,715 633,A01,2,7,2,0,1,1,1,NaN
94489,202200055301,715 631,A01,2,7,0,0,8,19,1,NaN
94490,202200055301,715 632,B01,2,7,0,2,1,1,1,NaN
94491,202200055302,715 629,A01,1,33,0,2,1,1,1,NaN


Pareil ici, comme on peut le voir on retrouve `id_vehicule` et `num_veh`. Donc pareil, on peut supprimer tout ça.

On peut aussi observer que `occutc` est null la très grande majorité du temps puisque cette variable ne concerne que les transports en commun. Il y a trop de valeurs nulle pour qu'on puisse vraiement tirer quelques choses de concluant avec cette feature. Donc autant la supprimer.

In [441]:
print(f"Nombre de valeurs nulles dans `occutc` : {data["vehicules"]["occutc"].isna().sum()} soit {data["vehicules"]["occutc"].isna().sum()/94493}%")


Nombre de valeurs nulles dans `occutc` : 93676 soit 0.9913538568994529%


In [442]:
data["vehicules"].drop(columns=["num_veh", "occutc", "Num_Acc"], inplace=True)
# Change the type of every column to int8 except the first one.
data["vehicules"] = data["vehicules"].astype({col: 'int8' for col in data["vehicules"].columns if col != "id_vehicule"})
# Check what we've done.
data["vehicules"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94493 entries, 0 to 94492
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_vehicule  94493 non-null  object
 1   senc         94493 non-null  int8  
 2   catv         94493 non-null  int8  
 3   obs          94493 non-null  int8  
 4   obsm         94493 non-null  int8  
 5   choc         94493 non-null  int8  
 6   manv         94493 non-null  int8  
 7   motor        94493 non-null  int8  
dtypes: int8(7), object(1)
memory usage: 1.4+ MB


### Lieux

Passon à notre troisième fichier: les lieux

In [443]:
data["lieux"]

,Num_Acc,catr,voie,v1,v2,circ,nbv,vosp,prof,pr,pr1,plan,lartpc,larrout,surf,infra,situ,vma
0,202200000001,4,TEIL(vieille route du),0,NaN,2,2,0,1,(1),(1),1,NaN,-1,1,0,1,50
1,202200000002,4,NaN,0,NaN,2,2,0,1,(1),(1),1,NaN,-1,1,0,1,50
2,202200000003,3,ROND POINT DE BREZILLET,0,NaN,-1,2,0,1,0,0,1,NaN,-1,1,5,1,50
3,202200000004,4,QUATORZE JUILLET (RUE DU),0,NaN,1,1,0,2,(1),(1),1,NaN,4,1,0,1,30
4,202200000005,3,ROUTE DE JEAN MOULIN-RN 538,0,NaN,2,2,0,1,8,0,1,NaN,-1,1,0,1,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55297,202200055298,3,71,-1,NaN,2,2,0,2,(1),(1),1,NaN,-1,1,0,8,80
55298,202200055299,3,973,-1,NaN,2,2,0,1,29,0,2,NaN,-1,1,0,3,80
55299,202200055300,3,22,0,D,2,2,0,1,39,553,2,NaN,-1,7,0,3,80
55300,202200055301,3,18,-1,D,2,2,0,1,30,125,1,NaN,-1,1,0,1,80


In [444]:
pp(data["lieux"].info())
pp(data["lieux"].describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55302 entries, 0 to 55301
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Num_Acc  55302 non-null  int64 
 1   catr     55302 non-null  int64 
 2   voie     50497 non-null  object
 3   v1       55302 non-null  int64 
 4   v2       5255 non-null   object
 5   circ     55302 non-null  int64 
 6   nbv      55302 non-null  object
 7   vosp     55302 non-null  int64 
 8   prof     55302 non-null  int64 
 9   pr       55302 non-null  object
 10  pr1      55302 non-null  object
 11  plan     55302 non-null  int64 
 12  lartpc   28 non-null     object
 13  larrout  55302 non-null  object
 14  surf     55302 non-null  int64 
 15  infra    55302 non-null  int64 
 16  situ     55302 non-null  int64 
 17  vma      55302 non-null  int64 
dtypes: int64(11), object(7)
memory usage: 7.6+ MB
None
            Num_Acc          catr            v1          circ          vosp  \
count  5.530200e

In [445]:
(Counter(data["lieux"]["lartpc"]))

Counter({nan: 55274,
         '3': 6,
         '2': 5,
         '5': 3,
         '3,5': 2,
         '2,5': 2,
         '7': 1,
         '3,1': 1,
         '2,8': 1,
         '5,25': 1,
         '10': 1,
         '1,5': 1,
         '5,5': 1,
         '0,4': 1,
         '3,2': 1,
         '10,5': 1})

In [446]:
(Counter(data["lieux"]["nbv"]))

Counter({'2': 20311,
         2: 13854,
         '4': 3793,
         '1': 3362,
         4: 2735,
         '3': 2550,
         1: 2280,
         3: 1749,
         '6': 854,
         '0': 648,
         6: 568,
         '5': 541,
         0: 451,
         5: 395,
         ' -1': 328,
         '8': 256,
         -1: 233,
         8: 157,
         7: 52,
         '7': 51,
         '10': 42,
         10: 22,
         9: 21,
         '9': 19,
         12: 11,
         '12': 9,
         11: 6,
         '11': 3,
         '#ERREUR': 1})

In [447]:
Counter(data["lieux"]["nbv"].apply( lambda x : int(x) if x!="#ERREUR" else -1 ))

Counter({2: 34165,
         4: 6528,
         1: 5642,
         3: 4299,
         6: 1422,
         0: 1099,
         5: 936,
         -1: 562,
         8: 413,
         7: 103,
         10: 64,
         9: 40,
         12: 20,
         11: 9})

Dans le fichier `lieux` , en se référant à la doc, on voit que `voie` `V1` `V2` `pr` et `pr1` sont totalement inutiles puisque les valeurs ne sont pas forcément uniques pour chaque route (en effet, les numéro de voies, les adresses et autres peuvent être communs à plusieurs lieux). Je vais donc les supprimer également.

De plus, pour `lartpc` qui définie la largeur du terre-plein central, n'est quasiment jamais défini. Et le peux de fois où il l'est , il ne représente pas assez de data pour pouvoir en tirer quoi que ce soit. Je vais donc le supprimer.

`larrout` on peut convertir cela en centimetre afin d'avoir une valeur entiere et non un flottant, 

Enfin, comme nous pouvons le voir, il va y avoir un peu de boulot à faire pour gérer correctement les valeurs de la feature `nbv`

In [448]:
def convert_to_centimetre(x: str) -> int:
    # This could've been done in a single line in the lambda function, but for the sake of readability I'm gonna write it in a separate function.
    x = float(x.replace(',', '.'))
    eps = 1e-4
    if -1-eps < x < -1+eps:
        return -1
    return int(x*100)

In [449]:
# Drop the unwanted features
data["lieux"].drop(columns=["voie", "v1", "v2", "pr", "pr1", "lartpc"], inplace=True)
# Convert the width of the road to centimeters and integer type
data["lieux"]["larrout"] = data["lieux"]["larrout"].apply(lambda x : convert_to_centimetre(x) ).astype('int16')
# Correct the datatype of nbv
data["lieux"]["nbv"] = data["lieux"]["nbv"].apply( lambda x : int(x) if x!="#ERREUR" else -1 )
# Change the type of every column to int8 except the first one and larrout (because it can take values larger than 128.
data["lieux"] = data["lieux"].astype({col: 'int8' for col in data["lieux"].columns if col not in ('Num_Acc', "larrout")})
data["lieux"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55302 entries, 0 to 55301
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Num_Acc  55302 non-null  int64
 1   catr     55302 non-null  int8 
 2   circ     55302 non-null  int8 
 3   nbv      55302 non-null  int8 
 4   vosp     55302 non-null  int8 
 5   prof     55302 non-null  int8 
 6   plan     55302 non-null  int8 
 7   larrout  55302 non-null  int16
 8   surf     55302 non-null  int8 
 9   infra    55302 non-null  int8 
 10  situ     55302 non-null  int8 
 11  vma      55302 non-null  int8 
dtypes: int16(1), int64(1), int8(10)
memory usage: 1.1 MB


### Carcteristiques

Et enfin, il ne reste plus que le fichier de caractéristiques à traiter.

In [450]:
data["carcteristiques"]

,Accident_Id,jour,mois,an,hrmn,lum,dep,com,agg,int,atm,col,adr,lat,long
0,202200000001,19,10,2022,16:15,1,26,26198,2,3,1,3,TEIL(vieille route du),"44,5594200000","4,7257200000"
1,202200000002,20,10,2022,08:34,1,25,25204,2,3,1,3,Miranda,"46,9258100000","6,3462000000"
2,202200000003,20,10,2022,17:15,1,22,22360,2,6,1,2,ROND POINT DE BREZILLET,"48,4931620000","-2,7604390000"
3,202200000004,20,10,2022,18:00,1,16,16102,2,3,8,6,LOHMEYER (RUE),"45,6926520000","-0,3262900000"
4,202200000005,19,10,2022,11:45,1,13,13103,1,1,1,2,ROUTE DE JEAN MOULIN-RN 538,"43,6755790366","5,0927031775"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55297,202200055298,1,1,2022,03:50,3,2B,2B293,1,1,1,6,D71,"42,3101650000","9,4785830000"
55298,202200055299,1,1,2022,07:20,3,84,84074,1,1,1,6,D973,"43,7531640000","5,2244760000"
55299,202200055300,1,1,2022,04:27,3,74,74001,1,1,9,6,D22,"46,2825320000","6,7328060000"
55300,202200055301,1,1,2022,08:40,1,81,81099,1,3,1,3,Chemin Toulze,"43,9272650000","1,9156370000"


In [451]:
data["carcteristiques"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55302 entries, 0 to 55301
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Accident_Id  55302 non-null  int64 
 1   jour         55302 non-null  int64 
 2   mois         55302 non-null  int64 
 3   an           55302 non-null  int64 
 4   hrmn         55302 non-null  object
 5   lum          55302 non-null  int64 
 6   dep          55302 non-null  object
 7   com          55302 non-null  object
 8   agg          55302 non-null  int64 
 9   int          55302 non-null  int64 
 10  atm          55302 non-null  int64 
 11  col          55302 non-null  int64 
 12  adr          54069 non-null  object
 13  lat          55302 non-null  object
 14  long         55302 non-null  object
dtypes: int64(9), object(6)
memory usage: 6.3+ MB


In [452]:
# On pourra facilement supprimer l'année comme elle n'apporte aucune information.
print(Counter(data["carcteristiques"]["an"]))
# On pourra aussi transformer les heures/minutes en entier. On les regroupera par paquets de 12min pour avoir un range de valeur plus faible.
# Je prends le parti de réduire la précision de la mesure du temps à 12min près car bien que je sois d'accord que l'heure de l'accident
# puisse jouer sur la classification éventuellement, de là à dire qu'à la minute près ça peut jouer je ne pense pas.
# De plus, en divisant par 12, cela me permet de stocker cette valeur sur 8bit.
print(data["carcteristiques"]["hrmn"].nunique(), data["carcteristiques"]["hrmn"].values)
print( data["carcteristiques"]["hrmn"].apply( lambda x: (int(x[:2])*60 + int(x[3:]))//12 ).nunique() )
# Pareil pour les départements, afin de réduire le range de valeurs possible, je vais faire un mapping de chaque département de sortes à ce que tout les numéros soient contigue.
print( sorted(data["carcteristiques"]["dep"].unique()) )
# Pareil pour les numéro de commune, je vais procéder de meme.
print( data["carcteristiques"]["com"].nunique() )

Counter({2022: 55302})
1398 ['16:15' '08:34' '17:15' ... '04:27' '08:40' '16:55']
120
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '21', '22', '23', '24', '25', '26', '27', '28', '29', '2A', '2B', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '971', '972', '973', '974', '975', '976', '977', '978', '986', '987', '988']
11253


In [453]:
# Drop the unwanted features
data["carcteristiques"].drop(columns=["an", "adr", "lat", "long"], inplace=True)
# Let's convert time to integers approximating at 12 minutes
data["carcteristiques"]["hrmn"] = data["carcteristiques"]["hrmn"].apply( lambda x: (int(x[:2])*60 + int(x[3:]))//12 )
# Convert dep to contiguous numbers.
mapping_departement = { val: idx for idx, val in enumerate(sorted(data["carcteristiques"]["dep"].unique())) }
data["carcteristiques"]["dep"] = data["carcteristiques"]["dep"].map(mapping_departement)
# Same here. Convert com to contiguous numbers.
mapping_commune = { val: idx for idx, val in enumerate(sorted(data["carcteristiques"]["com"].unique())) }
data["carcteristiques"]["com"] = data["carcteristiques"]["com"].map(mapping_commune).astype('int16')
# Rename the column Accident_Id in order to match other tables
data["carcteristiques"].rename(columns={"Accident_Id":"Num_Acc"}, inplace=True)
# Change the type of every column to int8 except the first one and com (because it can take values larger than 128.
data["carcteristiques"] = data["carcteristiques"].astype({col: 'int8' for col in data["carcteristiques"].columns if col not in ('Num_Acc', 'com')})
data["carcteristiques"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55302 entries, 0 to 55301
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Num_Acc  55302 non-null  int64
 1   jour     55302 non-null  int8 
 2   mois     55302 non-null  int8 
 3   hrmn     55302 non-null  int8 
 4   lum      55302 non-null  int8 
 5   dep      55302 non-null  int8 
 6   com      55302 non-null  int16
 7   agg      55302 non-null  int8 
 8   int      55302 non-null  int8 
 9   atm      55302 non-null  int8 
 10  col      55302 non-null  int8 
dtypes: int16(1), int64(1), int8(9)
memory usage: 1.0 MB


Maintenant qu'on s'est occupé de ça, on peut juste join toutes les données.

In [454]:
merged_data = data["usagers"]
merged_data

,Num_Acc,id_vehicule,place,catu,grav,sexe,age,trajet,secu1,secu2,secu3,locp,actp,etatp
0,202200000001,813 952,1,1,3,1,14,5,2,8,-1,-1,-1,-1
1,202200000001,813 953,1,1,1,1,74,5,1,8,-1,-1,-1,-1
2,202200000002,813 950,1,1,4,1,34,9,1,0,-1,0,0,-1
3,202200000002,813 951,1,1,1,1,52,4,1,0,-1,0,0,-1
4,202200000003,813 948,1,1,1,1,20,0,1,0,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126657,202200055301,715 631,1,1,1,2,20,5,1,-1,-1,0,0,-1
126658,202200055301,715 631,8,2,3,2,18,5,1,-1,-1,0,0,-1
126659,202200055301,715 632,1,1,4,2,69,5,1,-1,-1,0,0,-1
126660,202200055302,715 629,1,1,3,1,30,1,2,6,-1,-1,-1,-1


In [455]:
merged_data = merged_data.merge(data["vehicules"], on="id_vehicule")
# Now that we don't need the vehicule id anymore, we can drop it
merged_data.drop(columns=["id_vehicule"], inplace=True)
merged_data

,Num_Acc,place,catu,grav,sexe,age,trajet,secu1,secu2,secu3,locp,actp,etatp,senc,catv,obs,obsm,choc,manv,motor
0,202200000001,1,1,3,1,14,5,2,8,-1,-1,-1,-1,1,2,0,2,1,9,1
1,202200000001,1,1,1,1,74,5,1,8,-1,-1,-1,-1,1,7,0,2,2,1,1
2,202200000002,1,1,4,1,34,9,1,0,-1,0,0,-1,2,7,0,2,8,15,1
3,202200000002,1,1,1,1,52,4,1,0,-1,0,0,-1,2,10,0,2,1,1,1
4,202200000003,1,1,1,1,20,0,1,0,-1,-1,-1,-1,2,7,0,2,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126657,202200055301,1,1,1,2,20,5,1,-1,-1,0,0,-1,2,7,0,0,8,19,1
126658,202200055301,8,2,3,2,18,5,1,-1,-1,0,0,-1,2,7,0,0,8,19,1
126659,202200055301,1,1,4,2,69,5,1,-1,-1,0,0,-1,2,7,0,2,1,1,1
126660,202200055302,1,1,3,1,30,1,2,6,-1,-1,-1,-1,1,33,0,2,1,1,1


In [456]:
merged_data = merged_data.merge(data["lieux"], on="Num_Acc")
merged_data

,Num_Acc,place,catu,grav,sexe,age,trajet,secu1,secu2,secu3,...,circ,nbv,vosp,prof,plan,larrout,surf,infra,situ,vma
0,202200000001,1,1,3,1,14,5,2,8,-1,...,2,2,0,1,1,-1,1,0,1,50
1,202200000001,1,1,1,1,74,5,1,8,-1,...,2,2,0,1,1,-1,1,0,1,50
2,202200000002,1,1,4,1,34,9,1,0,-1,...,2,2,0,1,1,-1,1,0,1,50
3,202200000002,1,1,1,1,52,4,1,0,-1,...,2,2,0,1,1,-1,1,0,1,50
4,202200000003,1,1,1,1,20,0,1,0,-1,...,-1,2,0,1,1,-1,1,5,1,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126657,202200055301,1,1,1,2,20,5,1,-1,-1,...,2,2,0,1,1,-1,1,0,1,80
126658,202200055301,8,2,3,2,18,5,1,-1,-1,...,2,2,0,1,1,-1,1,0,1,80
126659,202200055301,1,1,4,2,69,5,1,-1,-1,...,2,2,0,1,1,-1,1,0,1,80
126660,202200055302,1,1,3,1,30,1,2,6,-1,...,3,4,0,1,1,400,1,0,1,70


In [457]:
merged_data = merged_data.merge(data["carcteristiques"], on="Num_Acc")
merged_data

,Num_Acc,place,catu,grav,sexe,age,trajet,secu1,secu2,secu3,...,jour,mois,hrmn,lum,dep,com,agg,int,atm,col
0,202200000001,1,1,3,1,14,5,2,8,-1,...,19,10,81,1,24,2590,2,3,1,3
1,202200000001,1,1,1,1,74,5,1,8,-1,...,19,10,81,1,24,2590,2,3,1,3
2,202200000002,1,1,4,1,34,9,1,0,-1,...,20,10,42,1,23,2446,2,3,1,3
3,202200000002,1,1,1,1,52,4,1,0,-1,...,20,10,42,1,23,2446,2,3,1,3
4,202200000003,1,1,1,1,20,0,1,0,-1,...,20,10,86,1,20,2217,2,6,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126657,202200055301,1,1,1,2,20,5,1,-1,-1,...,1,1,43,1,81,9900,1,3,1,3
126658,202200055301,8,2,3,2,18,5,1,-1,-1,...,1,1,43,1,81,9900,1,3,1,3
126659,202200055301,1,1,4,2,69,5,1,-1,-1,...,1,1,43,1,81,9900,1,3,1,3
126660,202200055302,1,1,3,1,30,1,2,6,-1,...,1,3,84,1,41,4679,2,1,1,2


In [458]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126662 entries, 0 to 126661
Data columns (total 41 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   Num_Acc  126662 non-null  int64
 1   place    126662 non-null  int8 
 2   catu     126662 non-null  int8 
 3   grav     126662 non-null  int8 
 4   sexe     126662 non-null  int8 
 5   age      126662 non-null  int8 
 6   trajet   126662 non-null  int8 
 7   secu1    126662 non-null  int8 
 8   secu2    126662 non-null  int8 
 9   secu3    126662 non-null  int8 
 10  locp     126662 non-null  int8 
 11  actp     126662 non-null  int8 
 12  etatp    126662 non-null  int8 
 13  senc     126662 non-null  int8 
 14  catv     126662 non-null  int8 
 15  obs      126662 non-null  int8 
 16  obsm     126662 non-null  int8 
 17  choc     126662 non-null  int8 
 18  manv     126662 non-null  int8 
 19  motor    126662 non-null  int8 
 20  catr     126662 non-null  int8 
 21  circ     126662 non-null  int8 
 

In [459]:
merged_data.to_pickle("./data/2022/merged_data-2022.pkl")

In [460]:
testtruc = pd.read_pickle("./data/2022/merged_data-2022.pkl")
testtruc

,Num_Acc,place,catu,grav,sexe,age,trajet,secu1,secu2,secu3,...,jour,mois,hrmn,lum,dep,com,agg,int,atm,col
0,202200000001,1,1,3,1,14,5,2,8,-1,...,19,10,81,1,24,2590,2,3,1,3
1,202200000001,1,1,1,1,74,5,1,8,-1,...,19,10,81,1,24,2590,2,3,1,3
2,202200000002,1,1,4,1,34,9,1,0,-1,...,20,10,42,1,23,2446,2,3,1,3
3,202200000002,1,1,1,1,52,4,1,0,-1,...,20,10,42,1,23,2446,2,3,1,3
4,202200000003,1,1,1,1,20,0,1,0,-1,...,20,10,86,1,20,2217,2,6,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126657,202200055301,1,1,1,2,20,5,1,-1,-1,...,1,1,43,1,81,9900,1,3,1,3
126658,202200055301,8,2,3,2,18,5,1,-1,-1,...,1,1,43,1,81,9900,1,3,1,3
126659,202200055301,1,1,4,2,69,5,1,-1,-1,...,1,1,43,1,81,9900,1,3,1,3
126660,202200055302,1,1,3,1,30,1,2,6,-1,...,1,3,84,1,41,4679,2,1,1,2


In [462]:
testtruc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126662 entries, 0 to 126661
Data columns (total 41 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   Num_Acc  126662 non-null  int64
 1   place    126662 non-null  int8 
 2   catu     126662 non-null  int8 
 3   grav     126662 non-null  int8 
 4   sexe     126662 non-null  int8 
 5   age      126662 non-null  int8 
 6   trajet   126662 non-null  int8 
 7   secu1    126662 non-null  int8 
 8   secu2    126662 non-null  int8 
 9   secu3    126662 non-null  int8 
 10  locp     126662 non-null  int8 
 11  actp     126662 non-null  int8 
 12  etatp    126662 non-null  int8 
 13  senc     126662 non-null  int8 
 14  catv     126662 non-null  int8 
 15  obs      126662 non-null  int8 
 16  obsm     126662 non-null  int8 
 17  choc     126662 non-null  int8 
 18  manv     126662 non-null  int8 
 19  motor    126662 non-null  int8 
 20  catr     126662 non-null  int8 
 21  circ     126662 non-null  int8 
 